In [1]:
import pandas as pd
import glob
import datetime
import numpy as np
import os
os.getcwd()

'/home/jian/Projects/Saatva/Aggregating_Sales_2018'

In [2]:
list_weekly_sales_csv=glob.glob("/home/jian/Projects/Saatva/Aggregating_Sales_2018/sales_data_2018/*.csv")
sales_before_0524_xlsx="/home/jian/Projects/Saatva/Aggregating_Sales_2018/sales_data_2018/Sales_Jian_2018_5.24.2018.xlsx"


df_csv_files=pd.DataFrame({"file_path":list_weekly_sales_csv})
df_csv_files['download_date']=df_csv_files['file_path'].apply(lambda x: x.split("transactions_for_jubaplus-")[1][:8])
df_csv_files=df_csv_files.sort_values("download_date",ascending=False).reset_index()
del df_csv_files['index']

In [3]:
sales_before_0524=pd.read_excel(sales_before_0524_xlsx,dtype=str)
sales_before_0524=sales_before_0524[['Order_id','Date','Time','Zip_Code','Sales','Source','Phone_1','Phone_2','Brand']]

sales_before_0524=sales_before_0524.rename(columns={"Order_id":"Order ID","Zip_Code":"Shipping Zip","Sales":"Product Revenue","Source":"Order Type",
                                                   "Phone_1":"Phone 1","Phone_2":"Phone 2"})
sales_before_0524['Product Revenue']=sales_before_0524['Product Revenue'].astype(float)
sales_before_0524['Date']=sales_before_0524['Date'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())



In [4]:
df_sales_2018=pd.DataFrame(columns=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type',
                                              'Phone 1','Phone 2','Brand'])
for csv_file in df_csv_files['file_path'].tolist():
    df=pd.read_csv(csv_file,dtype=str,usecols=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type',
                                              'Phone 1','Phone 2','Brand'])
    df=df[pd.notnull(df['Order ID'])]
    try:
        df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
    except:
        df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%B-%Y").date())
    date_max=df['Date'].max()
    date_min=df['Date'].min()
    
    df=df[(df['Date']!=date_max) & (df['Date']!=date_min)]
    df=df[~df['Date'].isin(df_sales_2018['Date'].unique().tolist())]
    df_sales_2018=df_sales_2018.append(df)

In [5]:
sales_before_0524=sales_before_0524[~sales_before_0524['Date'].isin(df_sales_2018['Date'])]
df_sales_2018=df_sales_2018.append(sales_before_0524)

In [9]:
df_sales_2018['date']=df_sales_2018['Date'].astype(str).apply(lambda x: x.replace("-",""))

In [10]:
df_sales_2018.to_csv("/home/jian/Projects/Saatva/Aggregating_Sales_2018/Saatva_Aggregated_sales_2018_JL_20190517.csv",index=False)


In [8]:
df_sales_2018.head(2)

,Order ID,Date,Time,Shipping Zip,Product Revenue,Order Type,Phone 1,Phone 2,Brand,date
20,658363,2018-12-27,23:51:05,10013,1099.00,online,917-613-9302,NaN,Saatva,2018-12-27
21,658362,2018-12-27,23:44:03,20910,1099.00,phone,(301) 608-3646,(202) 439-6719,Saatva,2018-12-27


In [11]:
df_sales_2018.shape

(107281, 10)